In [5]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB
import numpy as np

import sys
sys.path.append('../../')
import saedfsc

In [6]:
numCustomersTypes = 3
numSubsystems = 10
numObjectives = 11

# Create a list of supplier names
names = ['CustomerType' + str(i) for i in range(1, numCustomersTypes + 1)]

qty = np.random.randint(low=25, high=1000, size=numCustomersTypes)
meanDemand = np.random.randint(low=15, high=30, size=numCustomersTypes)
stdDevDemand = np.random.randint(low=1, high=5, size=numCustomersTypes)
priceFocus = np.random.random(size=numCustomersTypes)

partworthWts = [np.array2string(np.random.random(numSubsystems), separator=',') for _ in range(numCustomersTypes)]
perfWts = [np.array2string(np.random.random(numObjectives), separator=',') for _ in range(numCustomersTypes)]

# Create the DataFrame
df = DataFrame({
    'Name': names,
    'Quantity': qty,
    'Mean Demand': meanDemand,
    'Std Dev Demand': stdDevDemand,
    'PriceFocus': priceFocus,
    'PartworthUtilityWeights': partworthWts,
    'PerformanceUtilityWeights': perfWts
})
df.to_csv('../resources/customers-{}.csv'.format(numCustomersTypes), index=False)
customers: DataFrame = read_csv('../resources/customers-{}.csv'.format(numCustomersTypes))
customers

,Name,Quantity,Mean Demand,Std Dev Demand,PriceFocus,PartworthUtilityWeights,PerformanceUtilityWeights
0,CustomerType1,785,16,3,0.943944,"[0.15911931,0.31794878,0.60614632,0.11971278,0...","[0.07353441,0.23812319,0.88198825,0.18071148,0..."
1,CustomerType2,127,21,3,0.392428,"[0.78091746,0.76884294,0.09853732,0.42630513,0...","[0.06854985,0.53773759,0.93934427,0.94031569,0..."
2,CustomerType3,966,20,1,0.324017,"[0.65919069,0.33134067,0.9477402 ,0.70901273,0...","[0.98918637,0.95977688,0.30505229,0.96212633,0..."


Print dictionary of customers and their weights.

In [7]:
name_weights_dict = customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}
cWts

{'CustomerType1': array([0.07353441, 0.23812319, 0.88198825, 0.18071148, 0.1822069 ,
        0.86919866, 0.12726251, 0.86094461, 0.06614307, 0.12788772,
        0.65358829]),
 'CustomerType2': array([0.06854985, 0.53773759, 0.93934427, 0.94031569, 0.3447028 ,
        0.36092742, 0.62696254, 0.68824787, 0.34874709, 0.25989053,
        0.19709851]),
 'CustomerType3': array([0.98918637, 0.95977688, 0.30505229, 0.96212633, 0.09427434,
        0.6756195 , 0.52721584, 0.61422457, 0.71667116, 0.61611324,
        0.86961674])}

In [8]:
cQty = dict(zip(customers['Name'], customers['Quantity'])) # customer quantities
cQty

{'CustomerType1': 785, 'CustomerType2': 127, 'CustomerType3': 966}